Versão "Beta" ainda, pois falta alterar a parte de extração de texto do pdfplumber para o PyMuPDF.

**Execução 1** - baixando e importanto bibliotecas utilizadas ao decorrer da execução do programa

In [ ]:
# Instalando as bibliotecas que serão utilizadas
try:
  !pip install requests
  !pip install beautifulsoup4
  !pip install pandas
  !pip install regex
  !pip install google-api-python-client
  !pip install urllib3
  !pip install pdfplumber
except: # Caso ocorra algum erro ao instalar os pacotes/bibliotecas seremos notificados com uma mensagem na tela
  print('\nErro ao instalar pacotes/bibliotecas.')
else:
  from googleapiclient.discovery import build
  from google.colab import auth
  from google.colab import drive
  import requests
  from bs4 import BeautifulSoup
  import pandas as pd
  import time
  import re
  from urllib import request
  import pdfplumber
  import os
  import sys
  execucao = True # Variável que armazenará o sucesso no download das bibliotecas
  print('='*100)
  print('Bibliotecas carregadas!\nPode executar o código abaixo.')
  print('='*100)

**Execução 2** - Conectando ambiente do Google Colab ao Google Drive

In [ ]:
try:
  drive.mount('/content/drive')
except:
  print('\nErro ao "sincronizar" Google Drive com este ambiente do Google Colab.\nExecute a célula novamente ou tente "montar" o Drive manualmente no botão de "Montar Drive" no menu lateral esquerdo.')
  print('Descrição do erro --> '+str(sys.exc_info()[0].__name__)+":",str(sys.exc_info()[1])+'\n')
  execucao = False
else:
  print('='*100)
  print('Drive conectado com sucesso.\nPode prosseguir na execução das próximas células.')
  print('='*100)

**Execução 3** - Definindo variáveis e funções que serão utilizadas ao decorrer do programa

In [ ]:
if execucao:
  headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
                (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"}
  timeout = 30
  try:
    def CriaDiretorio(caminho : str):
      os.makedirs(caminho)
      time.sleep(1)
      if os.path.isdir(caminho):
        print(f'Pasta {caminho} criada com sucesso.')
        return True
      else:
        print(f'Tentando criar pasta {caminho} novamente.')
        time.sleep(3)
        if os.path.isdir(caminho):
          print(f'Pasta {caminho} criada com sucesso.')
          return True
        else:
          print(f'Pasta {caminho} não foi carregada corretamente.')
          execucao = False
          return False
  except:
    print('Erro ao criar função de criação e verificação de diretório (pasta de trabalho).\nTente executar esta célula novamente.')
    print('Descrição do erro --> '+str(sys.exc_info()[0].__name__)+":",str(sys.exc_info()[1])+'\n')
  else:
    print('='*100)
    print('Variáveis para requisições e função de criação de diretório criadas com sucesso.\nPode dar continuidade na execução das células abaixo.')
    print('='*100)
else:
  print('\nFalha no carregamento das bibliotecas. Tente executar a primeira célula de códigos novamente.\n')

**Execução 4** - Criação de pasta "Coleções Repositorio UFSC" e sub-pasta "Links dos trabalhos - Repositorio UFSC" para armazenamento dos primeiros dados (links das coleções)

In [ ]:
if execucao:
  try:
    if(CriaDiretorio("/content/drive/MyDrive/Coleções Repositorio UFSC/Links dos trabalhos - Repositorio UFSC")):
      pass
    else:
      execucao = False
      print('\nErro na criação da pasta "/content/drive/MyDrive/Coleções Repositorio UFSC/Links dos trabalhos - Repositorio UFSC".\nTente novamente.\n')
  except:
    print('\nErro ao sincronizar e criar pasta no Drive com o código do Google Colab.\nTente executar a célula de códigos novamente.\n')
    print('Descrição do erro --> '+str(sys.exc_info()[0].__name__)+":",str(sys.exc_info()[1])+'\n')
    execucao = False
  else:
    print('='*100)
    print('Pastas de trabalhos "Coleções Repositorio UFSC" e "Links dos trabalhos - Repositorio UFSC" criadas com sucesso!\nPode continuar e executar os códigos abaixo.')
    print('='*100)
else:
  print('\nFalha no carregamento das bibliotecas. Tente executar a primeira célula de códigos novamente.\n')

**Execução 5** - Preenchimento de dados (links) das coleções e dos trabalhos em cada coleção

In [ ]:
if execucao:
  # Variável start armazenará o momento que o programa começar para visualizarmos, ao final,
  # quanto tempo levou para ser executado por completo
  start = time.time()

  # Criando dicionário que armazenará possíveis avisos e/ou erros
  lista_de_falhas = {'Avisos':[],'Erros':[]}

  # Primeiro, entraremos no link da comunidade de Dissertações e Teses, para tal armazenaremos o link na variável link
  link = 'https://repositorio.ufsc.br/handle/123456789/74645'

  # Passaremos um headers com "User-Agent" para identificarmos para o servidor o tipo de navegador
  # ou dispositivo que está fazendo a solicitação (boas práticas para não pensar que estamos com "más intenções")

  try:
    # Executando uma requisção para o link e armazenando o resultado na variável "site"
    site = requests.get(link,headers=headers,timeout=timeout)
  except:
    print(f'Ocorreu um erro durante a requisição com link "{link}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+":",str(sys.exc_info()[1])+'\n')
    lista_de_falhas['Erros'].append(f'Ocorreu um erro durante a requisição com link "{link}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
  else:
    # Obtendo a resposta do status da requisição (se for igual a 200 quer dizer que foi bem sucedida)
    res_code = site.status_code

    if res_code == 200: # Se o resultado do status code for 200, prosseguiremos com a execução
      numero_total_de_trabalhos = 0 # Criando uma variável que armazenará a quantidade de trabalhos captados
      dic = {'Colecoes':[],'Links colecoes':[]} # Criando dicionário que armazenará as informações de nome e link das coleções
      lista_links_trabalhos = []

      # Passando o conteúdo da requisição para um formato html da BeautifulSoup
      soup = BeautifulSoup(site.content, 'html.parser')

      # Encontrando o campo que contém as informações das coleções presentes na página
      campo_colecoes = soup.find('div', class_='ds-static-div secondary')
      if campo_colecoes != None: # Se a busca pelo campo for bem sucedida, prosseguimos
        colecoes = campo_colecoes.find_all('li') # Buscando todos os elementos (coleções) dentro do campo de coleções
        if len(colecoes) == 0: # Caso o identificador dos elementos tenha mudado, ocorrerá um erro e seremos notificados
          print('\nNão foi possível encontrar os elementos das coleções corretamente.')
          lista_de_falhas['Erros'].append('Não foi possível encontrar os elementos das coleções corretamente.')
        else:
          # Percorrendo todos os elementos de coleção
          for indice, colecao in enumerate(colecoes):
            nome_e_link_colecao = colecao.find('a', href=re.compile('/handle')) # Coletando o nome da coleção
            try:
              nome_colecao = nome_e_link_colecao.text
              dic['Colecoes'].append(nome_colecao)

              link_colecao = nome_e_link_colecao['href']
              link_colecao = 'https://repositorio.ufsc.br'+link_colecao
              dic['Links colecoes'].append(link_colecao)
            except:
              print(f'\nErro ao encontrar o nome e link da coleção número {indice} da lista.\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+":",str(sys.exc_info()[1])+'\n')
              lista_de_falhas['Avisos'].append(f'Erro ao encontrar o nome e link da coleção número {indice} da lista.\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
            # break # Só vai pegar a primeira coleção
            limite_colecoes = 5
            if indice >= limite_colecoes:
              print(f'\nLIMITE DE COLEÇÕES ({limite_colecoes}) ATINGIDO.')
              break

          print('\nNúmero de coleções encontradas:',len(dic['Links colecoes']))
          num_colecao = 0
          # Percorrendo a listagem de links das coleções
          for i in range(len(dic['Links colecoes'])):
            dic_trabalho = {'Links trabalhos':[]}
            num_colecao +=1

            # Enviaremos uma requisição para a página de busca em cada coleção, na primeira página de resultados com um
            # total de 100 resultados por página (padrão é 10, mas podemos alterar isso mexendo no link da requisição.
            # Escolhido foi 100 que é um número 10 vezes maior que o padrão e a resposta vem em, aproximadamente, 10s)
            link_colecao_atual = dic['Links colecoes'][i] + '/discover?rpp=5&etal=0&group_by=none&page=1'

        #                                       MUDAR O RPP = 5 PARA RPP = 100 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

            try:
              site = requests.get(link_colecao_atual,headers=headers,timeout=timeout)
            except:
              print(f'Ocorreu um erro durante a requisição com link "{link_colecao_atual}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+":",str(sys.exc_info()[1])+'\n')
              lista_de_falhas['Erros'].append(f'Ocorreu um erro durante a requisição com link "{link_colecao_atual}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
              break
            else:
              res_code = site.status_code
              if res_code == 200:
                soup = BeautifulSoup(site.content, 'html.parser')
                # Outro modo de encontrar objetos dentro do conteúdo html (agora o número de páginas)
                numero_paginas = soup.find('div', attrs={"class":'pagination-masked top'})
                if numero_paginas == None:
                  print('\nNão foi possível achar número de páginas corretamente.\n(numero_paginas == None)')
                  lista_de_falhas["Avisos"].append(f'Não foi possível achar número de páginas corretamente da coleção {dic["Colecoes"][i]} --> (numero_paginas == None).')
                else:
                  paginas_links = numero_paginas.find('ul',class_='pagination-links')
                  if paginas_links == None:
                    print('\nNão foi possível achar número de páginas corretamente.\n(paginas_links == None)')
                    lista_de_falhas["Avisos"].append(f'Não foi possível achar número de páginas corretamente da coleção {dic["Colecoes"][i]} --> (paginas_links == None).')
                  else:
                    links_pagina = paginas_links.find_all('li')
                    if links_pagina == None:
                      print('\nNão foi possível achar número de páginas corretamente.\n(links_pagina == None)')
                      lista_de_falhas["Avisos"].append(f'Não foi possível achar número de páginas corretamente da coleção {dic["Colecoes"][i]} --> (links_pagina == None).')
                    else:
                      try:
                        numero_paginas = int(links_pagina[len(links_pagina)-1].find('a').text)
                      except:
                        print('\nNúmero de páginas não pode ser convertido para inteiro, logo foi setado como 1.\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+":",str(sys.exc_info()[1])+'\n')
                        numero_paginas = 1
                        lista_de_falhas["Avisos"].append(f'Número de páginas da coleção {dic["Colecoes"][i]} não pode ser convertido para inteiro, logo foi setado como 1\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
                      print('\nNúmero total de páginas:',numero_paginas)

                      # Encontrando todos os links de teses e dissertações dentro do campo do resultado de busca dos mesmos
                      links = soup.find_all('li', class_=re.compile('ds-artifact-item clearfix'))
                      if len(links) == 0:
                        print('\nNão foi possível encontrar corretamente os links para os trabalhos.')
                        lista_de_falhas["Avisos"].append(f'Não foi possível encontrar/captar corretamente os links para os trabalhos da coleção {dic["Colecoes"][i]}.')
                      else:
                        pagina = 1
                        # Entraremos num loop que percorrerá todas as páginas de resultado da parte de busca da coleção
                        while(pagina <= numero_paginas):
                          print('\nIndo para página:',pagina)
                          link = dic['Links colecoes'][i] + f'/discover?rpp=5&etal=0&group_by=none&page={pagina}'
                #                                       MUDAR O RPP = 5 PARA RPP = 100 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
                          try:
                            site = requests.get(link,headers=headers,timeout=timeout)
                          except:
                            print(f'Ocorreu um erro durante a requisição com link "{link}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+":",str(sys.exc_info()[1])+'\n')
                            lista_de_falhas['Erros'].append(f'Ocorreu um erro durante a requisição com link "{link}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
                            break
                          else:
                            res_code = site.status_code
                            if res_code == 200:
                              soup = BeautifulSoup(site.content, 'html.parser')
                              links = soup.find_all('li', class_=re.compile('ds-artifact-item clearfix'))
                              if len(links) == 0:
                                print(f'\nNão foi possível encontrar corretamente os links dos resultados da coleção {dic["Colecoes"][i]} no link: "{link}".')
                                lista_de_falhas['Avisos'].append(f'Não foi possível encontrar/coletar corretamente os links dos resultados da coleção {dic["Colecoes"][i]} no link: "{link}".')
                              else:
                                for links_trabalho in links:
                                  link_trabalho = links_trabalho.find('a', href=re.compile('/handle'))
                                  if link_trabalho != None:
                                    link_trabalho = link_trabalho['href']
                                    link_trabalho = 'https://repositorio.ufsc.br'+link_trabalho
                                    dic_trabalho['Links trabalhos'].append(link_trabalho)
                                    numero_total_de_trabalhos += 1
                                  else:
                                    print(f'\nLink do trabalho não foi encontrado corretamente. Coleção: {dic["Colecoes"][i]}, link: "{link}".')
                                    lista_de_falhas['Avisos'].append(f'Link do trabalho não foi encontrado corretamente. Coleção: {dic["Colecoes"][i]}, link: "{link}".')
                            else:
                              print(f'\nErro de conexão com o site (internet fora do ar ou servidor do repositório fora do ar). Coleção que parou: {dic["Colecoes"][i]}.')
                              lista_de_falhas['Erros'].append(f'Erro de conexão com o site do repositório "{link}". Possivelmente internet instável/fora do ar ou servidor do repositório fora do ar). Coleção que parou: {dic["Colecoes"][i]}.')

                            # Limitando a busca (primeiramente não vamos passsar por todos os resultados)
                            limite_pagina = 1
                            if pagina >= limite_pagina:
                              print(f'\nLIMITE DE PÁGINAS ({limite_pagina}) ALCANÇADO.')
                              break
                            else:
                              pagina += 1

                        print('\nNúmero de links captados:',len(dic_trabalho["Links trabalhos"]))

                      df = pd.DataFrame(dic_trabalho)

                      # Tratando possíveis erros na hora de salvar o arquivo
                      if '/' in dic["Colecoes"][i]:
                        index = dic["Colecoes"][i].find('/')
                        dic["Colecoes"][i] = dic["Colecoes"][i][:index]+'_'+dic["Colecoes"][i][index+1:]
                      if ':' in dic["Colecoes"][i]:
                        index = dic["Colecoes"][i].find(':')
                        dic["Colecoes"][i] = dic["Colecoes"][i][:index]+' '+dic["Colecoes"][i][index+1:]
                      try:
                        df.to_csv(f'/content/drive/MyDrive/Coleções Repositorio UFSC/Links dos trabalhos - Repositorio UFSC/{dic["Colecoes"][i]}_Links_dos_Trabalhos.csv',index=False,encoding='utf-8',sep=',')
                      except:
                        print(f'\nErro ao fazer download do arquivo .csv para coleção {dic["Colecoes"][i]}.\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+":",str(sys.exc_info()[1])+'\n')
                        lista_de_falhas['Erros'].append(f'Erro ao fazer download do arquivo .csv para coleção {dic["Colecoes"][i]}.\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
                      else:
                        print(f'\nDownload do arquivo .csv da coleção {dic["Colecoes"][i]} feito com sucesso!\n')
                      # Código abaixo comentado, pois era apenas para fins de limitar o número de coleções que serão visitadas
                      # num_colecao_maxima = 4
                      # if num_colecao >= num_colecao_maxima:
                      #   print(f'Parou na {num_colecao} coleção.')
                      #   break
          print('='*100)
          print('Programa executado com sucesso!\nNúmero total de links de trabalhos coletados:',numero_total_de_trabalhos)
          print(f'Número de Coleções visitadas:',len(dic["Links colecoes"]))
          print('='*100)

      else: # Caso a localização ou identificação do campo de coleções tenha mudado, ocorrerá um erro e seremos notificados.
        print('\nCampo das coleções não identificado corretamente.')
        lista_de_falhas['Erros'].append("Campo das coleções 'find('div', class_='ds-static-div secondary')' não identificado corretamente.")

    else: # Caso a internet esteja ruim ou o servidor da UFSC esteja fora do ar, ocorrerá um erro e seremos notificados.
      print('\nErro de conexão com o site (internet fora do ar ou servidor do repositório fora do ar)...')
      lista_de_falhas['Erros'].append(f'Erro de conexão com o site do repositório "{link}". Possivelmente internet instável/fora do ar ou servidor do repositório fora do ar.')

  print('\n')
  print('='*100)
  print('Número de avisos gerados:',len(lista_de_falhas["Avisos"]))
  print('Número de erros gerados:',len(lista_de_falhas["Erros"]))
  print('='*100)
  # Salvando arquivos de Avisos e Erros ao decorrer desta execução
  if (len(lista_de_falhas['Avisos']) > 0):
    file_path = '/content/drive/MyDrive/Coleções Repositorio UFSC/Links dos trabalhos - Repositorio UFSC/1 AVISOS.txt'
    texto = 'AVISOS:\n\n'
    for i in range(len(lista_de_falhas['Avisos'])):
      texto = texto + '\n' + lista_de_falhas['Avisos'][i]
    with open(file_path, 'w') as f:
      f.write(texto)
      print('\nArquivo de avisos salvo com sucesso.\nVerifique-o!')
      f.close()
  if (len(lista_de_falhas['Erros']) > 0):
    file_path = '/content/drive/MyDrive/Coleções Repositorio UFSC/Links dos trabalhos - Repositorio UFSC/1 ERROS.txt'
    texto = 'ERROS:\n\n'
    for i in range(len(lista_de_falhas['Erros'])):
      texto = texto + '\n' + lista_de_falhas['Erros'][i]
    with open(file_path, 'w') as f:
      f.write(texto)
      print('\nArquivo de erros salvo com sucesso.\nVerifique-o!')
      f.close()

  end = time.time()

  duracao = end - start
  print('\n')
  print('='*100)
  print('Duração total da primeira execução:',round(duracao,2))
  print('='*100)
else:
  print('\nFalha na execução das células anteriores. Volte, leia as saídas das células anteriores e tente executá-las novamente.\n')

**Execução 6** - Entra link por link em cada coleção e coleta os metadados, entra no link do PDF, baixa o PDF, extrai texto do PDF, exclui arquivo PDF, criar arquivo .txt com o texto extraído e salva arquivo .txt.

In [ ]:
if execucao:
  start2 = time.time()
  try:
    arquivos_pasta_links = os.listdir('/content/drive/MyDrive/Coleções Repositorio UFSC/Links dos trabalhos - Repositorio UFSC')
    if not arquivos_pasta_links:
      print(f'\nA pasta "Links dos trabalhos - Repositorio UFSC" está vazia.\n')
      execucao = False
    else:
      print('\nArquivos de links foram encontrados na pasta "Links dos trabalhos - Repositorio UFSC".')
  except:
    print(f'\nErro ao carregar arquivos da pasta "Links dos trabalhos - Repositorio UFSC".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+":",str(sys.exc_info()[1])+'\n')
    execucao = False


  if execucao:
    for numeracao, arquivo in enumerate(arquivos_pasta_links):
      if (arquivo == '1 AVISOS.txt') or (arquivo == '1 ERROS.txt'):
        pass
      else:
        lista_de_falhas_trabalho = {'Erros':[],'Avisos':[]}
        try:
          df = pd.read_csv(f'/content/drive/MyDrive/Coleções Repositorio UFSC/Links dos trabalhos - Repositorio UFSC/{arquivo}',encoding='utf-8', sep=',')
        except:
          print('\nErro ao abrir arquivo dos Links dos trabalhos.\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+":",str(sys.exc_info()[1])+'\n')
          lista_de_falhas_trabalho['Erros'].append('Erro ao abrir arquivo dos Links dos trabalhos.\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
        else:
          lista_links_trabalhos = df["Links trabalhos"]
          dic = {'Título':[],'Autor':[],'Resumo':[],'Descrição':[],'Data':[],'Links':[],'Link do PDF':[]}

          for contagem, link in enumerate(lista_links_trabalhos):
            site = requests.get(link,headers=headers)
            res_code = site.status_code
            if res_code == 200:
              soup = BeautifulSoup(site.content, 'html.parser')
              tabela_info = soup.find('table', class_='ds-includeSet-table')
              tabela_arquivos = soup.find('table', attrs={"class":'ds-table file-list'})

              if (tabela_info == None) or (tabela_arquivos == None):
                print('\nNão foi possível encontrar corretamente as informações gerais do trabalho.')
                lista_de_falhas_trabalho['Avisos'].append(f'Não foi possível encontrar corretamente as informações gerais do trabalho com link: "{link}".\ntabela_info == None or tabela_arquivo == None.')
                dic["Data"].append('ERRO AO ENCONTRAR')
                dic["Descrição"].append('ERRO AO ENCONTRAR')
                dic["Link do PDF"].append('ERRO AO ENCONTRAR')
                dic["Resumo"].append('ERRO AO ENCONTRAR')
                dic["Texto do PDF"].append('ERRO AO ENCONTRAR')
                dic["Título"].append('ERRO AO ENCONTRAR')
                dic["Autor"].append('ERRO AO ENCONTRAR')
                dic["Links"].append(link)
              else:
                print('\nPegando informações gerais...')
                nao_eh_TCC = True
                titulo = autor = resumo = descricao = url = data = 'NÃO LOCALIZADO'
                for info in tabela_info:
                  analise = info.text
                  if 'Título' in analise[:(len('Título')+2)]:
                    titulo = analise
                    titulo = titulo[(len('titulo:')+2):].strip()
                  elif 'Autor' in analise[:len('Autor')+2]:
                    autor = analise
                    autor = autor[(len('autor:')+2):].strip()
                  elif 'Resumo' in analise[:len('Resumo')+2]:
                    resumo = analise
                    resumo = resumo[(len('resumo:')+2):].strip()
                  elif 'Descrição' in analise[:len('Descrição')+2]:
                    descricao = analise
                    descricao = descricao[(len('descricao:')+2):].strip()
                    if ('TCC' not in descricao[:len('TCC')+2]) and ('Trabalho de Conclusão de Curso' not in descricao[:len('Trabalho de Conclusão de Curso')+2]):
                      nao_eh_TCC = True
                    else:
                      nao_eh_TCC = False
                  elif ('URI' in analise[:len('URI')+2]) or ('URL' in analise[:len('URL')+2]):
                    url = analise
                    url = url[(len('url:')+2):].strip()
                  elif 'Data' in analise[:len('Data')+2]:
                    data = analise
                    data = data[(len('data:')+2):].strip()
                  else:
                    pass
                if nao_eh_TCC == True:
                  print('Pegando informações dos arquivos...')
                  for info_arq in tabela_arquivos:
                    analise = info_arq.text
                    if ('PDF' in analise) and ('.pdf' in analise):
                      print('\t--> Opa, esse aqui tem PDF...')
                      link_pdf = info_arq.find('a')
                      if 'href="' in str(link_pdf):
                        link_pdf = link_pdf['href']
                        link_pdf = 'https://repositorio.ufsc.br'+link_pdf

                        # Código para extração de texto de PDF (pdfplumber)
                        try:
                          index = arquivo.find('_Links_dos_Trabalhos.csv')
                          nome_arquivo = arquivo[:index]
                          local_PDF = False
                          if (CriaDiretorio(f'/content/drive/MyDrive/Coleções Repositorio UFSC/{nome_arquivo}/Trabalho {contagem}')):
                            local_PDF = True
                          else:
                            print(f'\nErro na hora de chamar a função para criação da pasta "/content/drive/MyDrive/Coleções Repositorio UFSC/{nome_arquivo}/Trabalho {contagem}".')
                            lista_de_falhas_trabalho['Avisos'].append(f'\nErro na hora de chamar a função para criação da pasta "/content/drive/MyDrive/Coleções Repositorio UFSC/{nome_arquivo}/Trabalho {contagem}".')
                        except:
                          print(f'\nErro na criação da pasta "/content/drive/MyDrive/Coleções Repositorio UFSC/{nome_arquivo}/Trabalho {contagem}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+":",str(sys.exc_info()[1])+'\n')
                          lista_de_falhas_trabalho['Avisos'].append(f'Erro na criação da pasta "/content/drive/MyDrive/Coleções Repositorio UFSC/{nome_arquivo}/Trabalho {contagem}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
                        else:
                          if local_PDF:
                            try:
                              caminho_arquivo = f'/content/drive/MyDrive/Coleções Repositorio UFSC/{nome_arquivo}/Trabalho {contagem}/arquivoPDF_temporario.pdf'
                              request.urlretrieve(link_pdf, caminho_arquivo)
                            except:
                              print(f'\nProblema ao baixar o PDF de {nome_arquivo}, trabalho {contagem}, com link "{link_pdf}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+":",str(sys.exc_info()[1])+'\n')
                              lista_de_falhas_trabalho['Avisos'].append(f'Problema ao baixar o PDF de {nome_arquivo}, trabalho {contagem}, com link "{link_pdf}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
                            else:
                              print(f'\nPDF temporário do trabalho {contagem} baixado com sucesso.')

                              try:
                                with pdfplumber.open(caminho_arquivo) as pdf:
                                  diferencas = []
                                  i = 1
                                  j = 1
                                  t = 0
                                  for pdf_page in pdf.pages:
                                    single_page_text = pdf_page.extract_text()
                                    try:
                                      primeiras_letras = re.findall(r'[0-9][0-9]|[0-9]',single_page_text[0:2].strip())
                                      if len(primeiras_letras) > 0:
                                        pagina_numerada = int(primeiras_letras[0])
                                        diferencas.append(pdf_page.page_number - pagina_numerada)
                                        i += 1
                                      ultimas_letras = re.findall(r'[0-9][0-9]|[0-9]',single_page_text[(len(single_page_text)-1)-2:].strip())
                                      if len(ultimas_letras) > 0:
                                        pagina_numerada = int(ultimas_letras[0])
                                        diferencas.append(pdf_page.page_number - pagina_numerada)
                                        j += 1
                                    except:
                                      pass
                                    else:
                                      t += 1
                                      single_page_text = single_page_text.strip()

                                    if i>=3:
                                      if (diferencas[len(diferencas)-1] == diferencas[len(diferencas)-2]) and (diferencas[len(diferencas)-2] == diferencas[len(diferencas)-3]):
                                        break
                                      else:
                                        i = 0
                                    elif j >=3:
                                      if (diferencas[len(diferencas)-1] == diferencas[len(diferencas)-2]) and (diferencas[len(diferencas)-2] == diferencas[len(diferencas)-3]):
                                        break
                                      else:
                                        j = 0
                                    elif t >= 20:
                                      diferencas.append(0)
                                      break

                                  diferenca = diferencas[len(diferencas)-1]

                                  all_text = ''
                                  for pdf_page in pdf.pages:
                                    single_page_text = pdf_page.extract_text()
                                    try:
                                      if single_page_text.startswith(str(pdf_page.page_number-diferenca)):
                                        # print('PAGINA COMEÇA COM NÚMERO DE PAGINAS')
                                        single_page_text = single_page_text[len(str(pdf_page.page_number-diferenca))+1:]
                                      elif single_page_text.endswith(str(pdf_page.page_number-diferenca)):
                                        # print('PAGINA TERMINA COM NÚMERO DE PAGINAS')
                                        single_page_text = single_page_text[:(len(single_page_text)-1)-2]
                                    except:
                                      pass
                                    else:
                                      single_page_text = single_page_text.replace('\n',' ').strip()

                                    all_text = all_text + '\n' + single_page_text

                                  # print(all_text)
                                  pdf.close()
                                  # tokens = list(all_text.lower().split())
                                  # print('\nTamanho da lista de tokens:',len(tokens))
                                try:
                                  os.remove(caminho_arquivo)
                                except:
                                  print(f'Problema ao excluir PDF temporário de {nome_arquivo}, trabalho {contagem}, com link "{link_pdf}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+":",str(sys.exc_info()[1])+'\n')
                                  lista_de_falhas_trabalho['Avisos'].append(f'Problema ao excluir PDF temporário de {nome_arquivo}, trabalho {contagem}, com link "{link_pdf}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
                                else:
                                  print('PDF temporário excluído com sucesso.')
                                try:
                                  with open(f"/content/drive/MyDrive/Coleções Repositorio UFSC/{nome_arquivo}/Trabalho {contagem}/PDF_texto.txt", "w") as arq:
                                    arq.write(all_text)
                                    arq.close()
                                except:
                                  print(f'Problema ao salvar o arquivo como .txt. do conteúdo do PDF temporário de {nome_arquivo}, trabalho {contagem}, com link "{link_pdf}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+":",str(sys.exc_info()[1])+'\n')
                                  lista_de_falhas_trabalho['Avisos'].append(f'Problema ao salvar o arquivo como .txt. do conteúdo do PDF temporário de {nome_arquivo}, trabalho {contagem}, com link "{link_pdf}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
                                else:
                                  print('Arquivo .txt com o texto do PDF criado com sucesso!')
                              except:
                                print(f'Problema ao abrir/ler o conteúdo do PDF de {nome_arquivo}, trabalho {contagem}, com link "{link_pdf}" e/ou salvar o arquivo como .txt.\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+":",str(sys.exc_info()[1])+'\n')
                                lista_de_falhas_trabalho['Avisos'].append(f'Problema ao abrir/ler o conteúdo do PDF de {nome_arquivo}, trabalho {contagem}, com link "{link_pdf}" e/ou salvar o arquivo como .txt.\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
                              for arquivo_temp in os.listdir(f'/content/drive/MyDrive/Coleções Repositorio UFSC/{nome_arquivo}/Trabalho {contagem}'):
                                if arquivo_temp.lower().endswith('.pdf'):
                                  caminho_do_arquivo = os.path.join(f'/content/drive/MyDrive/Coleções Repositorio UFSC/{nome_arquivo}/Trabalho {contagem}', arquivo_temp)
                                  os.remove(caminho_do_arquivo)
                                  print(f'Arquivo {arquivo_temp} removido forçadamente, após problema.')
                      try:
                        dic["Data"].append(data)
                      except:
                        dic["Data"].append('ERRO AO ENCONTRAR')
                        lista_de_falhas_trabalho["Avisos"].append(f'Problema ao encontrar DATA do trabalho com link "{link}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
                      try:
                        dic["Descrição"].append(descricao)
                      except:
                        dic["Descrição"].append('ERRO AO ENCONTRAR')
                        lista_de_falhas_trabalho["Avisos"].append(f'Problema ao encontrar DESCRIÇÃO do trabalho com link "{link}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
                      try:
                        dic["Link do PDF"].append(link_pdf)
                      except:
                        dic["Link do PDF"].append('ERRO AO ENCONTRAR')
                        lista_de_falhas_trabalho["Avisos"].append(f'Problema ao encontrar LINK DO PDF do trabalho com link "{link}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
                      try:
                        dic["Resumo"].append(resumo)
                      except:
                        dic["Resumo"].append('ERRO AO ENCONTRAR')
                        lista_de_falhas_trabalho["Avisos"].append(f'Problema ao encontrar RESUMO do trabalho com link "{link}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
                      try:
                        dic["Título"].append(titulo)
                      except:
                        dic["Título"].append('ERRO AO ENCONTRAR')
                        lista_de_falhas_trabalho["Avisos"].append(f'Problema ao encontrar TÍTULO do trabalho com link "{link}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
                      try:
                        dic["Autor"].append(autor)
                      except:
                        dic["Autor"].append('ERRO AO ENCONTRAR')
                        lista_de_falhas_trabalho["Avisos"].append(f'Problema ao encontrar AUTOR do trabalho com link "{link}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
                      try:
                        dic["Links"].append(link)
                      except:
                        dic["Links"].append('ERRO AO ENCONTRAR')
                        lista_de_falhas_trabalho["Avisos"].append(f'Problema ao encontrar LINK do trabalho com link "{link}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
                      # Estaremos pegando apenas o primeiro PDF !!!
                      # (alguns trabalhos tem mais de um PDF publicado,
                      # temos que analisar esse cenário com mais cautela ainda). Esse break é usado para isso..
                      break

                else:
                  print('\nEncontramos um TCC!\n\t--> Descartando arquivos...\n')

            else:
              print('\nErro de conexão com o site. Possivelmente internet instável/fora do ar ou servidor do repositório fora do ar.')
              lista_de_falhas_trabalho["Erros"].append(f'Erro de conexão com o site do repositório "{link}". Possivelmente internet instável/fora do ar ou servidor do repositório fora do ar.')

            if (len(lista_de_falhas_trabalho['Avisos']) > 0):
              file_path = f'/content/drive/MyDrive/Coleções Repositorio UFSC/{nome_arquivo}/Trabalho {contagem}/1 AVISOS.txt'
              texto = 'AVISOS:\n\n'
              for i in range(len(lista_de_falhas_trabalho['Avisos'])):
                texto = texto + '\n' + lista_de_falhas_trabalho['Avisos'][i]
              with open(file_path, 'w') as f:
                f.write(texto)
                print(f'\nArquivo de avisos salvo com sucesso para coleção {nome_arquivo} e trabalho {contagem}.\nVerifique-o!')
                f.close()
            if (len(lista_de_falhas_trabalho['Erros']) > 0):
              file_path = f'/content/drive/MyDrive/Coleções Repositorio UFSC/{nome_arquivo}/Trabalho {contagem}/1 ERROS.txt'
              texto = 'ERROS:\n\n'
              for i in range(len(lista_de_falhas_trabalho['Erros'])):
                texto = texto + '\n' + lista_de_falhas_trabalho['Erros'][i]
              with open(file_path, 'w') as f:
                f.write(texto)
                print(f'\nArquivo de erros salvo com sucesso para coleção {nome_arquivo} e trabalho {contagem}.\nVerifique-o!')
                f.close()

            # break # Só passará pelo primeiro link dos links de trabalhos
            if contagem >= 4: # Só passará pelos 5 primeiros links do arquivo que tem os links de trabalhos
              break

            contagem += 1

          # Criando avisos gerais na pasta de cada Coleção
          diretorio_raiz = f'/content/drive/MyDrive/Coleções Repositorio UFSC/{nome_arquivo}'
          pastas_com_arquivo_avisos = {}
          for diretorio_atual, subdiretorios, arquivos in os.walk(diretorio_raiz):
            if '1 AVISOS.txt' in arquivos:
              pasta_pai = os.path.basename(os.path.dirname(diretorio_atual))
              pasta_com_arquivo = os.path.basename(diretorio_atual)
              try:
                pastas_com_arquivo_avisos[str(pasta_pai)].append(pasta_com_arquivo)
              except:
                pastas_com_arquivo_avisos[str(pasta_pai)] = [pasta_com_arquivo]
          # print("Pastas que contém o arquivo '1 AVISOS.txt':")
          # print(pastas_com_arquivo_avisos)
          if (len(pastas_com_arquivo_avisos)> 0):
            file_path = f'/content/drive/MyDrive/Coleções Repositorio UFSC/{nome_arquivo}/1 AVISOS.txt'
            texto = 'AVISOS encontrados nos trabalhos:\n\n'
            for pasta in pastas_com_arquivo_avisos:
              for i in range(len(pastas_com_arquivo_avisos[pasta])):
                texto = texto + pastas_com_arquivo_avisos[pasta][i] + '\n'

            with open(file_path, 'w') as f:
              f.write(texto)
              print(f'\nArquivo de avisos gerais da coleção "{nome_arquivo}" salvo com sucesso.\nVerifique-o!')
              f.close()

          try:
            df = pd.DataFrame(dic)
            df.to_csv(f'/content/drive/MyDrive/Coleções Repositorio UFSC/{nome_arquivo}/Meta_dados_{numeracao}.csv', index=False, encoding='utf-8', sep=',')
          except:
            print(f'\nErro ao fazer download do arquivo .csv para coleção {nome_arquivo}.\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+":",str(sys.exc_info()[1])+'\n')
            lista_de_falhas_trabalho["Erros"].append(f'Erro ao fazer download do arquivo .csv para coleção {nome_arquivo}.\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
          else:
            print(f'\nDownload do arquivo .csv da coleção {nome_arquivo} feito com sucesso!')



  else:
    print('\nFalha ao abrir e carregar os arquivos presentes na pasta das coleções.\nTente novamente em alguns minutos.\n')


  end2 = time.time()
  duracao2 = end2 - start2
  print('\n')
  print('='*100)
  print('Duração total da segunda execução:',round(duracao2,2))
  print('='*100)

else:
  print('\nFalha no carregamento das bibliotecas e/ou na sincronização e criação de pasta do seu Google Drive. Tente executar a primeira e segunda células de códigos novamente.\n')

**Execução 7** - Final: Passa por todas as pastas criadas a fim de analisar se existem arquivos de avisos e erros e os informa no início das pastas "raízes"

In [ ]:
diretorio_raiz = '/content/drive/MyDrive/Coleções Repositorio UFSC'

pastas_com_arquivo_avisos = {}

for diretorio_atual, subdiretorios, arquivos in os.walk(diretorio_raiz):
  if '1 AVISOS.txt' in arquivos:
    pasta_pai = os.path.basename(os.path.dirname(diretorio_atual))
    pasta_com_arquivo = os.path.basename(diretorio_atual)
    try:
      pastas_com_arquivo_avisos[str(pasta_pai)].append(pasta_com_arquivo)
    except:
      pastas_com_arquivo_avisos[str(pasta_pai)] = [pasta_com_arquivo]

print("Pastas que contém o arquivo '1 AVISOS.txt':")
print(pastas_com_arquivo_avisos)

if (len(pastas_com_arquivo_avisos)> 0):
  file_path = f'/content/drive/MyDrive/Coleções Repositorio UFSC/1 AVISOS.txt'
  texto = 'AVISOS encontrados nas pastas:\n\n'
  for pasta in pastas_com_arquivo_avisos:
    texto = texto + pasta + '\n\n'
    # for i in range(len(pastas_com_arquivo_avisos[pasta])):
    #   texto = texto + '\n' + pastas_com_arquivo_avisos[pasta][i]
    # texto = texto + '\n\n'
  with open(file_path, 'w') as f:
    f.write(texto)
    print(f'\nArquivo de avisos gerais salvo com sucesso.\nVerifique-o!')
    f.close()


pastas_com_arquivo_erros = {}

for diretorio_atual, subdiretorios, arquivos in os.walk(diretorio_raiz):
  if '1 ERROS.txt' in arquivos:
    pasta_pai = os.path.basename(os.path.dirname(diretorio_atual))
    pasta_com_arquivo = os.path.basename(diretorio_atual)
    try:
      pastas_com_arquivo_erros[str(pasta_pai)].append(pasta_com_arquivo)
    except:
      pastas_com_arquivo_erros[str(pasta_pai)] = [pasta_com_arquivo]

print("\nPastas que contém o arquivo '1 ERROS.txt':")
print(pastas_com_arquivo_erros)

if (len(pastas_com_arquivo_erros)> 0):
  file_path = f'/content/drive/MyDrive/Coleções Repositorio UFSC/1 ERROS.txt'
  texto = 'ERROS encontrados nas pastas:\n\n'
  for pasta in pastas_com_arquivo_erros:
    texto = texto + pasta + '\n\n'
    # for i in range(len(pastas_com_arquivo_avisos[pasta])):
    #   texto = texto + '\n' + pastas_com_arquivo_avisos[pasta][i]
    # texto = texto + '\n\n'
  with open(file_path, 'w') as f:
    f.write(texto)
    print(f'\nArquivo de erros gerais salvo com sucesso.\nVerifique-o!')
    f.close()


pastas_com_arquivo_pdfs = {}

for diretorio_atual, subdiretorios, arquivos in os.walk(diretorio_raiz):
  for arquivo_pdf in arquivos:
    if arquivo_pdf.lower().endswith('.pdf'):
      pasta_pai = os.path.basename(os.path.dirname(diretorio_atual))
      pasta_com_arquivo = os.path.basename(diretorio_atual)
      try:
        pastas_com_arquivo_pdfs[str(pasta_pai)].append(pasta_com_arquivo)
      except:
        pastas_com_arquivo_pdfs[str(pasta_pai)] = [pasta_com_arquivo]
      # caminho_arquivo = os.path.join(diretorio_atual, arquivo_pdf)
      # os.remove(caminho_arquivo)

print("\nPastas que contém arquivo .pdf:")
print(pastas_com_arquivo_pdfs)

if (len(pastas_com_arquivo_pdfs)> 0):
  file_path = f'/content/drive/MyDrive/Coleções Repositorio UFSC/1 PDFs restantes.txt'
  texto = 'PDFs encontrados nas pastas:\n\n'
  for pasta in pastas_com_arquivo_pdfs:
    texto = texto + f'Pasta --> {pasta}'
    for i in range(len(pastas_com_arquivo_avisos[pasta])):
      texto = texto + '\n' + pastas_com_arquivo_avisos[pasta][i]
    texto = texto + '\n\n'
  with open(file_path, 'w') as f:
    f.write(texto)
    print(f'\nArquivo de erros gerais salvo com sucesso.\nVerifique-o!')
    f.close()